# GeoHash Legnth

GeoHash | Length |	Area Width 	Area Height
---|---|---
1 	| 5,009.4km 	| 4,992.6km
2 	| 1,252.3km 	| 624.1km
3 	| 156.5km 	| 156km
4 	| 39.1km | 	19.5km
5 	| 4.9km 	| 4.9km
6 	| 1.2km 	| 609.4m
7 	| 152.9m 	| 152.4m
8 	| 38.2m 	| 19m
9 	| 4.8m 	| 4.8m
10 	| 1.2m 	| 59.5cm
11 	| 14.9cm 	| 14.9cm
12 	| 3.7cm 	| 1.9cm

[source](http://incredible.ai/code-snippet/2019/03/16/GeoHash-And-GeoPandas/)

---

# GeoPandas json

In [1]:
{
    "type": "FeatureCollection",
    "features": [
        {
            "properties": {"name": "Alabama"},
            "id": "AL",
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [[[-87.359296, 35.00118], ...]]
                }
            },
        {
            "properties": {"name": "Alaska"},
            "id": "AK",
            "type": "Feature",
            "geometry": {
                "type": "MultiPolygon",
                "coordinates": [[[[-131.602021, 55.117982], ... ]]]
                }
            },
        ...
        ]
    }

{'type': 'FeatureCollection',
 'features': [{'properties': {'name': 'Alabama'},
   'id': 'AL',
   'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-87.359296, 35.00118], Ellipsis]]}},
  {'properties': {'name': 'Alaska'},
   'id': 'AK',
   'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-131.602021, 55.117982], Ellipsis]]]}},
  Ellipsis]}

---

# Converting

In [2]:
%load_ext autoreload
%autoreload 2

In [7]:
# old version of geohasing
import geohash

lat, lng = (37.497868971527424, 127.0276489936216) #  Lat, Lng
gangnam = geohash.encode(lat, lng, 5)
decoded_location = geohash.decode(gangnam)  # gangnam = 'wydm6'

print('Lat, Lng        :', lat, lng)
print('geohash     :', gangnam)
print('Decoded Location:', *decoded_location)

ModuleNotFoundError: No module named 'geohash'

In [8]:
import pygeohash as pgh

In [9]:
pgh.encode(latitude=42.6, longitude=-5.6)

'ezs42e44yx96'

In [12]:
pgh.encode(latitude=42.6, longitude=-5.6, precision=5)

'ezs42'

In [11]:
pgh.decode(geohash='ezs42')

(42.6, -5.6)

In [13]:
pgh.geohash_approximate_distance(geohash_1='bcd3u', geohash_2='bc83n')

625441

In [15]:
pgh.decode_exactly('wydm6')

(37.50732421875, 127.02392578125, 0.02197265625, 0.02197265625)

---

# Folium

In [48]:
import folium
import json
import geopandas as gpd
import numpy as np
from polygon_geohasher.polygon_geohasher import geohash_to_polygon

locations = [(37.49786897152, 127.02764899362),
             (37.50732421875, 126.95998046875),
             (37.50732421875, 127.06787109375),
             (37.62337890625, 127.02392578125),
             (37.44337890625, 126.97998046875),
             (37.43337890625, 127.06787109375),
             (37.55126953125, 127.52392578125),
             (37.55126953125, 126.939956046875),
             (37.55126953125, 127.06787109375)]

# Create Geo Pandas DataFrame
df = gpd.GeoDataFrame({'location':locations, 'value': np.random.rand(9)})
df['geohash'] = df['location'].apply(lambda l: pgh.encode(l[0], l[1], 6))
df['geometry'] = df['geohash'].apply(geohash_to_polygon)
df.crs = {'init': 'epsg:4326'}


print('features.properties.geohash')
display(json.loads(df.to_json())['features'][0])
display(df.head())

features.properties.geohash


/home/laia/.pyenv/versions/3.10.6/envs/lewagon_1014/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


{'id': '0',
 'type': 'Feature',
 'properties': {'location': [37.49786897152, 127.02764899362],
  'value': 0.740807982565238,
  'geohash': 'wydm6d'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[127.02392578125, 37.496337890625],
    [127.034912109375, 37.496337890625],
    [127.034912109375, 37.5018310546875],
    [127.02392578125, 37.5018310546875],
    [127.02392578125, 37.496337890625]]]}}

,location,value,geohash,geometry
0,"(37.49786897152, 127.02764899362)",0.740808,wydm6d,"POLYGON ((127.02393 37.49634, 127.03491 37.496..."
1,"(37.50732421875, 126.95998046875)",0.193010,wydm35,"POLYGON ((126.95801 37.50183, 126.96899 37.501..."
2,"(37.50732421875, 127.06787109375)",0.320135,wydm77,"POLYGON ((127.05688 37.50183, 127.06787 37.501..."
3,"(37.62337890625, 127.02392578125)",0.327242,wydq43,"POLYGON ((127.01294 37.62268, 127.02393 37.622..."
4,"(37.44337890625, 126.97998046875)",0.568070,wydm12,"POLYGON ((126.96899 37.44141, 126.97998 37.441..."


In [49]:
lat, lng = (37.497868971527424, 127.0276489936216) #  Lat, Lng
map = folium.Map((lat, lng), zoom_start=12, 
                 width=500, height=650, 
                  zoom_control=False,
                  scrollWheelZoom=False,
                  dragging=False,
                  tiles='cartodbdark_matter')
folium.Choropleth(geo_data=df, 
                  name='choropleth',
                  data=df,
                  columns=['geohash', 'value'],
                  key_on='feature.properties.geohash',
                  fill_color='YlGn',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name='asdf').add_to(map)

map

In [46]:
import branca.colormap as cm

def my_style_function(feature):
    label = feature['properties']['kmean_label']
    color = cm.step.Set1_09.scale(0, 9)(label)
    
    return dict(fillColor=color, 
                color='black', 
                weight=1.2, 
                fillOpacity=0.7)

df['kmean_label'] = np.arange(9)

lat, lng = (37.497868971527424, 127.0276489936216) # Lat, Lng
map = folium.Map((lat, lng), zoom_start=12, 
                 zoom_control=False, 
                 width=500, 
                 height=650,
                 scrollWheelZoom=False,
                 dragging=False,
                 tiles='cartodbpositron')
folium.GeoJson(df, style_function=my_style_function).add_to(map)
map

---

# Color maps

In [32]:
step = cm.StepColormap(
    ['green', 'yellow', 'red'],
    vmin=3, vmax=10,
    index=[3, 4, 8, 10],
    caption='step'
)

linear = folium.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=0, vmax=1
)

display(step)
display(linear)
display(cm.linear.Accent_03.scale(0, 24))
display(cm.linear.Accent_08)
display(cm.linear.Blues_03)

---

# Tiles


    - openstreetmap 
    - Stamen Toner 
    - cartodbdark_matter 
    - stamenwatercolor 
    - Mapbox Bright 
    - cartodbpositron 


In [39]:
m = folium.Map((37.5387343, 127.07511967), tiles='Stamen Toner', width=500, 
                 height=650,
                 scrollWheelZoom=False,
                 dragging=False)
folium.TileLayer('Stamen Toner').add_to(m)
m

In [41]:
m = folium.Map((37.53873434027448, 127.07511967328423), tiles='cartodbpositron',
               width=500, 
                 height=450,
                 scrollWheelZoom=False,
                 dragging=False)
m